In [4]:
import pandas as pd
import gurobipy as gb
from gurobipy import GRB

# Load the dataset
welders_data = pd.read_csv("/Users/huiyisang/Desktop/welders_data.csv")
welders_speed_data = pd.read_csv("/Users/huiyisang/Desktop/welders_speed_data.csv")

# Merge datasets on Welder_ID if necessary
if "Welder_ID" in welders_speed_data.columns:
    welders_data = welders_data.merge(welders_speed_data, on="Welder_ID", how="left")

# Extract relevant columns
welders_data = welders_data[["Welder_ID", "Safety_Rating", "Speed_Rating", 
                             "SMAW_Proficient", "GMAW_Proficient", 
                             "FCAW_Proficient", "GTAW_Proficient", "Experience_10_Years"]]

# Number of welders to hire
num_welders_to_hire = 16
num_welders = len(welders_data)

# Create Gurobi model
model = gb.Model("Welder Selection Without Speed-Safety Constraint")

# Decision variables
x = model.addVars(num_welders, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="WelderSelection")

# Objective function: Maximize the average speed rating
model.setObjective(gb.quicksum(welders_data.iloc[i]["Speed_Rating"] * x[i] for i in range(num_welders)) / num_welders_to_hire, GRB.MAXIMIZE)

# Constraints
model.addConstr(gb.quicksum(x[i] for i in range(num_welders)) == num_welders_to_hire, "TotalWelders")
model.addConstr(gb.quicksum(x[i] for i in range(num_welders) if welders_data.iloc[i]["SMAW_Proficient"] == 1 and welders_data.iloc[i]["GMAW_Proficient"] == 1) >= 0.5 * num_welders_to_hire, "SMAW_GMAW_Proficiency")
for technique in ["SMAW_Proficient", "GMAW_Proficient", "FCAW_Proficient", "GTAW_Proficient"]:
    model.addConstr(gb.quicksum(x[i] for i in range(num_welders) if welders_data.iloc[i][technique] == 1) >= 2, f"{technique}_Requirement")
model.addConstr(gb.quicksum(x[i] for i in range(num_welders) if welders_data.iloc[i]["Experience_10_Years"] == 1) >= 0.3 * num_welders_to_hire, "ExperienceRequirement")
model.addConstr(gb.quicksum(welders_data.iloc[i]["Safety_Rating"] * x[i] for i in range(num_welders)) / num_welders_to_hire >= 3.9, "SafetyRequirement")
model.addConstr(gb.quicksum(welders_data.iloc[i]["Speed_Rating"] * x[i] for i in range(num_welders)) / num_welders_to_hire >= 3.1, "SpeedRequirement")
num_hired_70_100 = gb.quicksum(x[i] for i in range(69, 100))
num_hired_101_130 = gb.quicksum(x[i] for i in range(100, 130))
model.addConstr(num_hired_70_100 >= 2 * num_hired_101_130 + 1, "HiringRange")

# **REMOVED: Constraint enforcing at least 3 welders with Speed = 1 and Safety >= 3**

# Solve the model
model.optimize()

# Store the new optimal objective function value
optimal_value_without_speed_safety_constraint = model.objVal
print(f"New Optimal Objective Function Value: {optimal_value_without_speed_safety_constraint}")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 144 columns and 891 nonzeros
Model fingerprint: 0xa831c116
Coefficient statistics:
  Matrix range     [6e-02, 2e+00]
  Objective range  [6e-02, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 0 rows and 9 columns
Presolve time: 0.00s
Presolved: 10 rows, 135 columns, 837 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.6875000e+00   4.590000e+01   0.000000e+00      0s
       7    3.4750000e+00   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.475000000e+00
New Optimal Objective Function Value: 3.475
